### Configuration of the environment

In [ ]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://bucket_code_completion" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.5MB 6.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-41fe7y1x
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-41fe7y1x
     |████████████████████████████████| 361 kB 7.1 MB/s 
     |████████████████████████████████| 65 kB 2.7 MB/s 
     |████████████████████████████████| 1.2 MB 9.4 MB/s 
     |████████████████████████████████| 3.4 MB 19.4 MB/s 
     |████████████████████████████████| 3.8 MB 77.1 MB/s 
     |████████████████████████████████| 1.8 MB 72.8 MB/s 
     |████████████████████████████████| 2.9 MB 78.7 MB/s 
     |████████████████████████████████| 883 kB 80.0 MB/s 
  Created wheel for t5: filename=t5-0.8.1-py3-none-any.whl size=219997 sha256=656a7b8c251c68c14b9858b7d791e730db2bda6634465897

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of the tsv files
We loaded the 6 tsv files, please be sure to upload them on the bucket and to copy the correct path

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_construct.tsv',
}

num_nq_examples_java_construct = dict(train=750000, validation=104037)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_construct.tsv',
}

num_nq_examples_android_construct = dict(train=750000, validation=98793)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_block.tsv',
}

num_nq_examples_java_block = dict(train=298470, validation=38840)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_block.tsv',
}

num_nq_examples_android_block = dict(train=204580, validation=26503)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_token.tsv',
}

num_nq_examples_java_token = dict(train=750000, validation=214682)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_token.tsv',
}

num_nq_examples_android_token = dict(train=750000, validation=198281)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for HP tuning.

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://bucket_code_completion/T5_extension/code.model'
vocab_path = 'gs://bucket_code_completion/T5_extension/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

JAVA CONSTRUCT

In [ ]:
def nq_java_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_java_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds



print("A few raw train examples...")
for ex in tfds.as_numpy(nq_java_construct("train").take(5)):
  print(ex)


A few raw train examples...
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound { Argument[] params = new Argument[2]; params[0] = new Argument("_this", "ManagedObjectReference", _this); params[1] = new Argument("users", "String[]", users); getWsc().invoke( <extra_id_0>); }', 'output': b'"UpdateLockdownExceptions", params, null'}
{'input': b'@Override public Collection<AgentProjectInfo> collectDependencies(String folder) { if ( <extra_id_0>){ devDependencies = findDevDependencies(folder); } File yarnLock = new File(folder + fileSeparator + YARN_LOCK); boolean yarnLockFound = yarnLock.isFile(); Collection<DependencyInfo> dependencies = new ArrayList<>(); if (yarnLockFound){ dependencies = parseYarnLock(yarnLock); } else { npmLsFailureStatus = true; } return getSingleProjectList(dependencies); }', 'output': b'!includeDevDependencies'}
{'input': b'@Override public Coll

In [ ]:
def java_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_construct')
t5.data.TaskRegistry.add(
    "java_construct",
    dataset_fn=nq_java_construct,
    splits=["train", "validation"],
    text_preprocessor=[java_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_CONSTRUCT:@Override public void doSessionCreated(ManagementContext managementContext, ServiceManagementBean serviceBean, IoSessionEx session, ManagementSessionType managementSessionType) throws Exception { SessionManagementBean sessionBean = managementContext.addSessionManagementBean(serviceBean, session); serviceBean.storeSessionManagementBean( <extra_id_0>); managementContext.getManagementSessionStrategy().doSessionCreated(sessionBean); super.doSessionCreated(managementContext, serviceBean, session, managementSessionType); }', 'inputs': array([    3,     2,  7589,    14,  5438, 19728,    55,  1569,    24,
          12,    18,   351,   454,  2111,     5,  3656,    86, 14364,
          86,     9,     3,   119,  3656,   517,   366,   517,     9,
           3, 20902,  2938,   552,     9, 11976,   454,    58, 14364,
         454,    58,     8,    42,   150,     7,  3491,  3656,   517,
         552,   517,    11, 14364,

JAVA TOKEN

In [ ]:
def nq_java_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_token("train").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault <extra_id_0> Argument[] params = new Argument[2]; params[0] = new Argument("_this", "ManagedObjectReference", _this); params[1] = new Argument("users", "String[]", users); getWsc().invoke("UpdateLockdownExceptions", params, null); }', 'output': b', UserNotFound {'}
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound { Argument[] params <extra_id_0> params[0] = new Argument("_this", "ManagedObjectReference", _this); params[1] = new Argument("users", "String[]", users); getWsc().invoke("UpdateLockdownExceptions", params, null); }', 'output': b'= new Argument[2];'}
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPer

In [ ]:
def java_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_token')
t5.data.TaskRegistry.add(
    "java_token",
    dataset_fn=nq_java_token,
    splits=["train", "validation"],
    text_preprocessor=[java_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(1)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'JAVA_TOKEN:public static void main(String[] args) throws RunnerException { Options opt = new OptionsBuilder() .include(".*" + SlidingTimeWindowReservoirsBenchmark.class.getSimpleName() + ".*") .warmupIterations(10) .measurementIterations(10) .addProfiler(GCProfiler.class) .measurementTime(TimeValue.seconds(3)) .timeUnit(TimeUnit.MICROSECONDS) .mode(Mode.AverageTime) .forks(1) .build(); new Runner(opt).run <extra_id_0> }', 'inputs': array([    3,     2,  7589,    14,  2956,    55,  5407,    50,    18,
        1014,     5,    31,    57,   309,     8,    42,     3,  2021,
          41,     7,     3,   408,  4499,    11,    23,     3,   408,
         160,    16,     3,     4,  3234,    29,     4,     2,    87,
          32,     3, 10269, 23759, 29207,    22,  5209,     4,    94,
           4,  2068,    16,    32,    28,     4,     2,   155,     3,
           4, 31343, 10836, 17908,     3,     4, 22542, 10836, 17908,
           3,     4,    76, 14510,     5,  5897,

JAVA BLOCK

In [ ]:
def nq_java_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public Object createField(String name) { Object reader = _fieldMap.get(name); if (reader == null) <extra_id_0> return reader; }', 'output': b'reader = NullFieldDeserializer.DESER;'}
{'input': b'@Nonnull public static JSInvocation invoke (@Nonnull final JQueryInvocation aJQueryInvocation, @Nonnull final JSAssocArray aOptions) <extra_id_0>', 'output': b'{ return invoke (aJQueryInvocation).arg (aOptions); }'}
{'input': b'public static String presentMinMaxCount(long minmax) { if (minmax == Long.MAX_VALUE || minmax == Long.MIN_VALUE) <extra_id_0> return String.valueOf(minmax); }', 'output': b'{ return UNDEF_STRING; }'}
{'input': b'@Override public NonBottomTypeNode<ElkClass, ElkNamedIndividual> getCreateNode( final Collection<? extends ElkClass> members) <extra_id_0>', 'output': b'{ return getCreateUpdateableTypeNode( classTaxonomy_.getCreateNode(members)); }'}
{'input': b'public static JsonException typeMismatch(Object indexOrName, Object actual, Str

In [ ]:
def java_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_block')
t5.data.TaskRegistry.add(
    "java_block",
    dataset_fn=nq_java_block,
    splits=["train", "validation"],
    text_preprocessor=[java_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'JAVA_BLOCK:@Override public void comment( char[] ch, int start, int length ) throws SAXException { try { startNode(XmlLexicon.COMMENT, XmlLexicon.COMMENT); currentNode.setProperty(XmlLexicon.COMMENT_CONTENT, String.valueOf(ch, start, length).trim()); endNode(); } catch (RepositoryException e) <extra_id_0> }', 'inputs': array([    3,     2,  7589,    14,  3834,    55,  1569,    24,    12,
          18,  2449,     5,   804,    57,  2053,     9,    34,   214,
           9,    34,   607,     3,     8,    42,  2261,     2,    41,
           7,    93,     7,   214,   176,     5,     2,   844, 19662,
           4, 14448,     9,     3,     2,   844, 19662,     4, 14448,
          10,     3, 15102,     4,  1974,     5,     2,   844, 19662,
           4, 14448,    14,   575,     9,    25,     4,   544,     5,
        1542,     9,   214,     9,   607,     8,     4,  1176,    45,
         429,   176,    19,     6,    95,    17,   628,    41,    62,
           8, 32099,   

ANDROID CONSTRUCT

In [ ]:
def nq_android_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_android_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_android_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'private void writeToFile(final String content) { if ( <extra_id_0>) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch (Exception e) { e.printStackTrace(); } }', 'output': b'!WRITE_TO_FILE'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch ( <extra_id_0>) { e.printStackTrace(); } }', 'output': b'Exception e'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File( <extra_id_0>); f.delete(); FileWriter fstream = new FileW

In [ ]:
def android_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_construct')
t5.data.TaskRegistry.add(
    "android_construct",
    dataset_fn=nq_android_construct,
    splits=["train", "validation"],
    text_preprocessor=[android_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'ANDROID_CONSTRUCT:private void init() { setPadding(0, Screen.dp(8), 0, Screen.dp( <extra_id_0>)); setWillNotDraw(false); bottomDrawable = getResources().getDrawable(R.drawable.card_shadow_bottom); topDrawable = getResources().getDrawable(R.drawable.card_shadow_top); }', 'inputs': array([    3, 13174,    14,  5438, 19728,    55,  9037,    18,   390,
          16,     7,     3, 10963,   443,     3,  1115,     4,  3631,
           5,     2,     8,     9,   204,     3,  1115,     4,  3631,
           5, 32099,    79, 27721,     5,   289,    10,  2913,  1153,
          11,  3363,    43,  4001,     5,   107,     4,  1398,     4,
        2805,    14, 17840,    14,  3809,    10,  1503,  1153,    11,
        3363,    43,  4001,     5,   107,     4,  1398,     4,  2805,
          14, 17840,    14,  2063,    10,     6,     1], dtype=int32), 'targets_pretokenized': b'8', 'targets': array([3, 2, 1], dtype=int32)}
{'inputs_pretokenized': b'ANDROID_CONSTRUCT:public static St

ANDROID TOKEN

In [ ]:
def nq_android_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void register(Context context <extra_id_0> IntentFilter intentFilter = buildFilter(); try { register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver maybe already registered, unregistering to register again."); unregister(context); register(context, intentFilter); } }', 'output': b') {'}
{'input': b'public void register(Context context) { IntentFilter intentFilter <extra_id_0> try { register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver maybe already registered, unregistering to register again."); unregister(context); register(context, intentFilter); } }', 'output': b'= buildFilter();'}
{'input': b'public void register(Context context) { IntentFilter intentFilter = buildFilter(); try <extra_id_0> register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver ma

In [ ]:
def android_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_token')
t5.data.TaskRegistry.add(
    "android_token",
    dataset_fn=nq_android_token,
    splits=["train", "validation"],
    text_preprocessor=[android_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'ANDROID_TOKEN:public void registerReceiver(Context context) { if (!isNetworkChangedListener) { isNetworkChangedListener = true; final ConnectivityManager connectivityManager = (ConnectivityManager) context.getSystemService(Context.CONNECTIVITY_SERVICE); NetworkInfo info = connectivityManager.getActiveNetworkInfo(); activeNetInfo = (info == null) ? 0 : info.toString().hashCode(); IntentFilter filter = new IntentFilter(); filter.addAction("android.net.conn.CONNECTIVITY_CHANGE"); context.registerReceiver(this <extra_id_0> } }', 'inputs': array([    3, 13174,    14,  2956,    55,  5407,    18,     3,  4190,
           5,    86,   105,     8,     7,    20,   125,   118,  1138,
        3991,     8,     7,    61,  1138,  3991,    11,    74,    13,
          48, 13008,   137, 22037,   137,    11,    17,  6751,   137,
           8,   105,     4,  1480,     5,    86,     4, 13756,    14,
         977,    10, 20473,   484,    11, 22037,   137,     4, 23588,
         145,

ANDROID BLOCK

In [ ]:
def nq_android_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT))<extra_id_0> if(Gdx.input.isKeyPressed(Keys.RIGHT)){ bucket.x += 200 * Gdx.graphics.getDeltaTime(); } if (bucket.x < 0) bucket.x = 0; if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'{ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); }'}
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT)){ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); } if(Gdx.input.isKeyPressed(Keys.RIGHT))<extra_id_0> if (bucket.x < 0) bucket.x = 0; if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'{ bucket.x += 200 * Gdx.graphics.getDeltaTime(); }'}
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT)){ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); } if(Gdx.input.isKeyPressed(Keys.RIGHT)){ bucket.x += 200 * Gdx.graphics.getDeltaTime(); } if (bucket.x < 0) <extra_id_0> if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'bucket.x = 0;'}
{'input':

In [ ]:
def android_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_block')
t5.data.TaskRegistry.add(
    "android_block",
    dataset_fn=nq_android_block,
    splits=["train", "validation"],
    text_preprocessor=[android_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_BLOCK:public void onViewCreated(View view, Bundle savedInstanceState) { view.findViewById(R.id.submit_color).setOnClickListener(new OnClickListener() { @Override public void onClick(View v) { onColorChosen(mColor); dismiss(); } }); view.findViewById(R.id.cancel).setOnClickListener(new OnClickListener() { @Override public void onClick(View v) <extra_id_0> }); }', 'inputs': array([    3, 13174,    14,  3834,    55,  5407,    18,     3, 14496,
           5,   108,   266,     9,   881,  1294,   116,     8,     7,
         266,     4,   994,     5,   107,     4,   104,     4,  4638,
          14,  1155,     8,     4,    63,  2516,     5,    84,     3,
        2516,    16,     7,    21,    24,    12,    18,   693,     5,
         108,   221,     8,     7,   147,   456,  9711,     5, 17375,
          10,  3637,    19,     6,     6,    10,   266,     4,   994,
           5,   107,     4,   104,     4,  1268,     8,     4

### Hyper Parameter tuning
You can run the HP tuning using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in), *PATH_GIN_FILE* (the gin file configuration for this HP tuning) and *PRETRAINED_DIR* (the folder that contains the pretrained model).  
**Keep attention** to change the *pretrained_model_dir* in finetune step (if you are starting the HP tuning from scratch you have to set the value *PRETRAINED_DIR*, if you are restarting the HP tuning from a previous saved checkpoint you have to set the value *MODEL_DIR*)

In [ ]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["java_construct", "java_token", "java_block", "android_construct", "android_token", "android_block"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_code_completion/T5_extension/HP_TUNING/slanted/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_code_completion/T5_extension/pretrained_with_masking'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = slanted_triangular,
    sequence_length={"inputs": 256, "targets": 256},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
PATH_GIN_FILE = 'gs://bucket_code_completion/T5_extension/HP_TUNING/slanted/operative_config.gin'
import gin

with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    #RUN FINE-TUNING
    FINETUNE_STEPS = 100000
    model.finetune(
        mixture_or_task_name="all_tasks",
        pretrained_model_dir=PRETRAINED_DIR,
        finetune_steps=FINETUNE_STEPS
    )

INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/HP_TUNING/slanted/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/HP_TUNING/slanted/operative_config.gin
INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/pretrained_with_masking/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/pretrained_with_masking/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_comment_completion/Matteo/HP_TUNING/slanted/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.82.119.210:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.82.119.210:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.82.119.210:8470', '_evaluation_master': 'grpc

/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('batch', 'batch'), ('heads', 'model'), ('ensemble', 'ensemble'), ('d_ff', 'model'), ('vocab', 'model'), ('experts', 'batch')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f7fc06ffef0>
INF

### Evaluate the performances

In [ ]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = 512
model.eval(
    mixture_or_task_name="all_tasks",
    # mixture_or_task_name="all_tasks",
    checkpoint_steps=-1 #evaluate only last checkpoint
)